# Two guys meet

This example is intended to illustrate the use uf `Sim.jl`.

In [1]:
using Sim, Printf

Since we implement the encounter as state machines, we need some definitions. For simplicity we don't implement a state here but only events.

In [2]:
struct Guy
    name
end

abstract type Encounter end
struct Meet <: Encounter
    someone
end
struct Greet <: Encounter
    num
    from
end
struct Response <: Encounter
    num
    from
end

We have some convenience functions:

In [3]:
comm = ("Nice to meet you!", "How are you?", "Have a nice day!", "bye bye")
say(name, n) =  @printf("%5.2f s, %s: %s\n", now(sim), name, comm[n])

say (generic function with 1 method)

Now we implement the actions between the state machines as different methods of a `step!`-function, which will be called according to events. 

In [6]:
function step!(me::Guy, σ::Meet)
    event!(sim, SimFunction(step!, σ.someone, Greet(1, me)), after, 2*rand())
    say(me.name, 1)
end

function step!(me::Guy, σ::Greet)
    if σ.num < 3
        event!(sim, SimFunction(step!, σ.from, Response(σ.num, me)), after, 2*rand())
        say(me.name, σ.num)
    else
        say(me.name, 4)
    end
end

function step!(me::Guy, σ::Response)
    event!(sim, SimFunction(step!, σ.from, Greet(σ.num+1, me)), after, 2*rand())
    say(me.name, σ.num+1)
end

step! (generic function with 3 methods)

Now we need a simulation clock and two guys (state machines), define a starting event and run it.

In [7]:
sim = Clock()
foo = Guy("Foo")
bar = Guy("Bar")

event!(sim, SimFunction(step!, foo, Meet(bar)), at, 10*rand())
run!(sim, 20)

 7.01 s, Foo: Nice to meet you!
 7.09 s, Bar: Nice to meet you!
 7.63 s, Foo: How are you?
 8.08 s, Bar: How are you?
 9.82 s, Foo: Have a nice day!
11.43 s, Bar: bye bye
Finished: 6 events, simulation time: 20.0
